In [ ]:
!pip install dnspython python-whois retrying colorama #install
#import
import dns.resolver
import whois
import retrying
import datetime
import ssl
import socket
import requests
import webbrowser

def get_mx_records(domain): #ent
    try:
        resolver = dns.resolver.Resolver(configure=False)
        resolver.nameservers = ['8.8.8.8']  # Change to the preferred DNS resolver IP address

        mx_records = resolver.resolve(domain, 'MX')
        return [mx.exchange.to_text() for mx in mx_records]
    except dns.exception.DNSException as e:
        print(f"Error retrieving MX records: {e}")

    return []

def get_spf_record(domain):
    try:
        resolver = dns.resolver.Resolver(configure=False)
        resolver.nameservers = ['8.8.8.8']  # Change to the preferred DNS resolver IP address

        txt_records = resolver.resolve(domain, 'TXT')
        for txt_record in txt_records:
            if 'v=spf1' in txt_record.to_text():
                return txt_record.to_text()
    except dns.exception.DNSException as e:
        print(f"Error retrieving SPF record: {e}")

    return None

def get_dmarc_record(domain):
    try:
        resolver = dns.resolver.Resolver(configure=False)
        resolver.nameservers = ['8.8.8.8']  # Change to the preferred DNS resolver IP address

        txt_records = resolver.resolve(f'_dmarc.{domain}', 'TXT')
        for txt_record in txt_records:
            if 'v=DMARC1' in txt_record.to_text():
                return txt_record.to_text()
    except dns.exception.DNSException as e:
        print(f"Error retrieving DMARC record: {e}")

    return None

def get_website_creation_date(domain):
    try:
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]  # Consider only the first creation date if there are multiple
        return creation_date
    except whois.parser.PywhoisError as e:
        print(f"Error retrieving website creation date: {e}")

    return None

def check_ssl_certificate(domain):
    try:
        context = ssl.create_default_context()
        with socket.create_connection((domain, 443)) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as ssock:
                certificate = ssock.getpeercert()
                ssl_expiry = certificate['notAfter']
                return ssl_expiry
    except (socket.gaierror, ssl.SSLError) as e:
        print(f"Error checking SSL certificate: {e}")
####    ssl_expiry = check_ssl_certificate(domain)
####    if ssl_expiry is not None:
####      print(f"SSL Expiry: {ssl_expiry}")

    return None

def retrieve_records(domain):
    print(f"Domain: {domain}")

    mx_records = get_mx_records(domain)
    if mx_records:
        mx_records_str = " ".join(mx_records)
        print(f"MX Records: {mx_records_str}")

    spf_record = get_spf_record(domain)
    if spf_record is not None:
        print(f"SPF Record: {spf_record}")

    dmarc_record = get_dmarc_record(domain)
    if dmarc_record is not None:
        print(f"DMARC Record: {dmarc_record}")

    creation_date = get_website_creation_date(domain)
    if creation_date is not None:
        print(f"Website Creation Date: {creation_date}")

def scan_domain(domain):
    query = f'domain:{domain}'

    # Construct the URL for the API request
    url = f'https://urlscan.io/api/v1/search/?q={query}'

    try:
        # Send an HTTP GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Check if there are any results
            if 'results' in data and isinstance(data['results'], list) and len(data['results']) > 0:
                results = data['results'][:1] # this 1 could be changed to get more results
                urls = [result['task']['url'] for result in results]
                screenshot_urls = [result['screenshot'] for result in results]

                # Print the URLs and open the screenshots in the browser
                for url, screenshot_url in zip(urls, screenshot_urls):
                    print("scan on URLSCAN.io:", url)
                    print("Screenshot URL:", screenshot_url)
                    # Open the screenshot URL in the browser
                    webbrowser.open(screenshot_url)
                    print("----")
            else:
                print("No URLs found.")
        else:
            print("Failed to scan the domain.")
            print("Error:", response.status_code)
    except requests.RequestException as e:
        print("Failed to scan the domain.")
        print("Error:", str(e))

#servicenow
servicenow_url_base_url = "https://pon.service-now.com/nav_to.do?uri=%2F$sn_global_search_results.do%3Fsysparm_search%3D"
servicenow_url = servicenow_url_base_url + domain

#Config
##Dmarc color
import re
from colorama import Fore, Style
def retrieve_records(domain):
    print(f"{Fore.GREEN}Domain:{Style.RESET_ALL} {domain}")
##Get_records
    mx_records = get_mx_records(domain)
    if mx_records:
        mx_records_str = " ".join(mx_records)
        print(f"{Fore.CYAN}MX Records:{Style.RESET_ALL} {mx_records_str}")
##get_SPF
    spf_record = get_spf_record(domain)
    if spf_record is not None:
        print(f"{Fore.CYAN}SPF Record:{Style.RESET_ALL} {spf_record}")
##get_Dmarc
    dmarc_record = get_dmarc_record(domain)
    if dmarc_record is not None:
        p_value = re.search(r'p=(\w+)', dmarc_record)
        if p_value:
            p_value = p_value.group(1)
            if p_value == "reject":
                dmarc_record = dmarc_record.replace(p_value, f"{Fore.GREEN}{p_value}{Style.RESET_ALL}")
            elif p_value == "quarantine":
                dmarc_record = dmarc_record.replace(p_value, f"{Fore.YELLOW}{p_value}{Style.RESET_ALL}")
            else:
                dmarc_record = dmarc_record.replace(p_value, f"{Fore.RED}{p_value}{Style.RESET_ALL}")
        print(f"{Fore.CYAN}DMARC Record:{Style.RESET_ALL} {dmarc_record}")
##Get_creation_date
    creation_date = get_website_creation_date(domain)
    if creation_date is not None:
        print(f"{Fore.CYAN}Website Creation Date:{Style.RESET_ALL} {creation_date}")
##Get_ssl_expiry
    ssl_expiry = check_ssl_certificate(domain)
    if ssl_expiry is not None:
        print(f"{Fore.CYAN}SSL Expiry:{Style.RESET_ALL} {ssl_expiry}")

In [ ]:
# Input
domain = input("Enter the domain name: ")
retrieve_records(domain)                      #DNS records
scan_domain(domain)                           ###Urlscan.io